# aladin 

사진, 제목, 저자, 가격

In [40]:
import requests
import urllib.request as req
from bs4 import BeautifulSoup as bs
import re
import json
base='https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page='


In [41]:
def save_img(src,title):
    pattern=r'[가-힣a-zA-Z0-9]+'
    result=re.findall(pattern,title)
    result=' '.join(result)
    path=f'data/images/{result}.jpg'
    req.urlretrieve(src,path)
    print(path,'저장됨')


In [61]:
books=[]
for i in range(1,3):
    url=base+str(i)
    res=requests.get(url)
    soup=bs(res.text,'html.parser')

    book_list=soup.findAll('div', {'class':'ss_book_list'})
    for index, book in enumerate(book_list):
        if index % 2==0:
            print(f'index:{index*(i+1)}')
            book_ul=book.find('ul')
            title=book_ul.find('a',{'class':'bo3'})
            if title !=None:
                title=title.text
            else:
                title="---"
            print(f'title:{title}')
            book_price=""
            book_author=""

            book_li_list=book_ul.findAll('li')
            price=book_li_list[3].find('span').text.replace(',','')
            print(f'price: {price}')
            author=book_li_list[2].find('a')
            if author !=None:
                author=author.text
            else:
                author=book_li_list[1].find('a').text

            img=book.select_one('img')
            src=img['src']
            save_img(title,src)
            print(img)

            print(f'author: {author}')
            books.append((title, int(price), author))

index:0
title:소년이 온다
price: 15000


TypeError: 'NoneType' object is not subscriptable

In [58]:
import pandas as pd
df=pd.DataFrame(books, columns=['title', 'price', 'author'])
df.to_csv('books.csv', index=False)
df

,title,price,author
0,소년이 온다,15000,한강
1,스토너,16800,존 윌리엄스
2,그릿 Grit,18800,김주환
3,봄밤의 모든 것,17000,백수린
4,양심,18000,최재천
...,...,...,...
95,패턴과 구조 영문법,19000,윤규철
96,Brown 일반화학 (일반화학교재연구회),55000,Theodore L. Brown
97,히카루가 죽은 여름 5,6500,모쿠모쿠렌
98,대온실 수리 보고서,18000,김금희


In [60]:
import pymysql
conn=pymysql.connect(
    host='localhost',
    user='G',
    password='1234',
    db='pydb',
    charset='utf8')
print(conn)
cursor=conn.cursor()
sql="insert into tblbook(title,price,author) values(%s,%s,%s)"
cursor.executemany(sql,books)
conn.commit()
conn.close()